In [1]:
import sys
sys.path.append('..')
from polynomial_program import PolynomialProgram

In [2]:
import functools
from collections import OrderedDict

# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import I, X, Y, Z
from qiskit.aqua.operators.list_ops import SummedOp
from qiskit.circuit.library import RealAmplitudes

In [3]:
M = [2, 6, 3]
K = [1, 4, 2]
T = [12, 6, 24]
d = 19


def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m


time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [4]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0


optimal_key = "0000001010001"


def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0

    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        key = key[::-1]
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1

    print('most likely solution: ', sample_most_likely(dict_res))
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)


def is_correct(key):
    return solution_vector_correct(key) and execution_time(key) == d


correct_machines = ['100', '010', '001']
machine_to_index = {'100': 0, '010': 1, '001': 2}


def solution_vector_correct(vector):
    task1_machine = vector[0:3]
    task2_machine = vector[3:6]
    task3_machine = vector[6:9]

    return task1_machine in correct_machines \
        and task2_machine in correct_machines \
        and task3_machine in correct_machines


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:3])
    task2_machine = machine_to_index.get(k[3:6])
    task3_machine = machine_to_index.get(k[6:9])

    
    task1_time = time_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_time = time_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_time = time_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    slack_sum = int(k[9]) * 8 + int(k[10]) * 4 + int(k[11]) * 2 + int(k[12]) * 1

    return task1_time + task2_time + task3_time + slack_sum


def execution_cost(k):
    task1_machine = machine_to_index.get(k[0:3])
    task2_machine = machine_to_index.get(k[3:6])
    task3_machine = machine_to_index.get(k[6:9])

    
    task1_cost = cost_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_cost = cost_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_cost = cost_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    return task1_cost + task2_cost + task3_cost


def incorrect_machine_count(k):
    task1_machine = machine_to_index.get(k[0:3])
    task2_machine = machine_to_index.get(k[3:6])
    task3_machine = machine_to_index.get(k[6:9])


    return (0 if k[0:3] in correct_machines else 1) \
         + (0 if k[3:6] in correct_machines else 1) \
         + (0 if k[6:9] in correct_machines else 1)

In [5]:
def get_cost_model(x):
    return sum(
        sum([
            cost_matrix[machine_index, task_index] * x[task_index * 3 + machine_index] 
            for machine_index 
            in range(3)
        ])
        for task_index 
        in range(3)
    )


def get_machine_usage_model(x):
    return sum(
        (1 - sum([x[task_index * 3 + machine_index] for machine_index in range(3)])) ** 2
        for task_index 
        in range(3)
    )

def get_deadline_model(x):
    time_sum = sum(
        sum([
            time_matrix[machine_index, task_index] * x[task_index * 3 + machine_index] 
            for machine_index 
            in range(3)
        ])
        for task_index 
        in range(3)
    )
    slack_sum = 8 * x[9] + 4 * x[10] + 2 * x[11] + x[12]
    time_constraint = (d - time_sum - slack_sum) ** 2
    
    return time_constraint

In [6]:
def compute_eigenvalues(quibit_op):
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 2 ** qubit_op.num_qubits
    eigensolver = NumPyEigensolver(qubit_op, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print('state\t\ttime\tcost\tmachine use\tcorrect\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenstate = eigenstate[::-1]
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{execution_time(eigenstate)}\t{execution_cost(eigenstate)}', end='')
        print(f'\t{incorrect_machine_count(eigenstate)}\t\t{is_correct(eigenstate)}\t{eigenvalue}')

In [7]:
A = 1
B = 20
C = 10

In [8]:
pp = PolynomialProgram(13)

pp.add_objective(get_cost_model(pp.x), A)
pp.add_objective(get_machine_usage_model(pp.x), B)
pp.add_objective(get_deadline_model(pp.x), C)

In [9]:
qubit_op, offset = pp.to_ising()

In [10]:
compute_eigenvalues(qubit_op)

state		time	cost	machine use	correct	eigenvalue
1000101000000	19.0	22.0	0		True	(-933.9999999999999+0j)
0011001000000	19.0	23.0	0		True	(-932.9999999999995+0j)
0101001000010	19.0	23.0	0		True	(-932.9999999999991+0j)
0010011000001	19.0	24.0	0		True	(-931.9999999999995+0j)
0010101000010	19.0	24.0	0		True	(-931.9999999999993+0j)
0100101000100	19.0	24.0	0		True	(-931.9999999999993+0j)
0100011000011	19.0	24.0	0		True	(-931.9999999999991+0j)
1001000010010	19.0	25.0	0		True	(-930.9999999999995+0j)
1001000100110	19.0	25.0	0		True	(-930.9999999999991+0j)
1000100101000	19.0	26.0	0		True	(-929.9999999999995+0j)
1000100010100	19.0	26.0	0		True	(-929.9999999999995+0j)
1000010100111	19.0	26.0	0		True	(-929.9999999999993+0j)
1000010010011	19.0	26.0	0		True	(-929.9999999999993+0j)
0011000101000	19.0	27.0	0		True	(-928.9999999999995+0j)
0011000010100	19.0	27.0	0		True	(-928.9999999999995+0j)
0101000101010	19.0	27.0	0		True	(-928.9999999999995+0j)
0101000010110	19.0	27.0	0		True	(-928.9999999999995+0j)


1101000010001	12.0	19.0	1		False	(-892.9999999999998+0j)
0101001000100	21.0	23.0	0		False	(-892.9999999999995+0j)
1001100001000	14.0	6.0	2		False	(-892.9999999999995+0j)
0001010010110	14.0	16.0	2		False	(-892.9999999999995+0j)
0001010101010	14.0	16.0	2		False	(-892.9999999999995+0j)
0011001000010	21.0	23.0	0		False	(-892.9999999999995+0j)
1001010001001	15.0	6.0	2		False	(-892.9999999999995+0j)
0001100010111	15.0	16.0	2		False	(-892.9999999999995+0j)
1011000100001	8.0	19.0	1		False	(-892.9999999999995+0j)
1011000100011	10.0	19.0	1		False	(-892.9999999999995+0j)
0101001000000	17.0	23.0	0		False	(-892.9999999999995+0j)
0101000110010	7.0	11.0	1		False	(-892.9999999999995+0j)
1101000100101	12.0	19.0	1		False	(-892.9999999999995+0j)
0011000110000	7.0	11.0	1		False	(-892.9999999999995+0j)
0001100101011	15.0	16.0	2		False	(-892.9999999999995+0j)
1001100001010	16.0	6.0	2		False	(-892.9999999999993+0j)
1101000100011	10.0	19.0	1		False	(-892.9999999999991+0j)
1001010000111	13.0	6.0	2		False	(-892

0010100110001	6.0	12.0	1		False	(-881.9999999999995+0j)
0100010110010	6.0	12.0	1		False	(-881.9999999999995+0j)
0010100110011	8.0	12.0	1		False	(-881.9999999999995+0j)
0000110011001	17.0	16.0	2		False	(-881.9999999999995+0j)
0010010110010	8.0	12.0	1		False	(-881.9999999999995+0j)
0000110010111	15.0	16.0	2		False	(-881.9999999999995+0j)
0000110101101	17.0	16.0	2		False	(-881.9999999999993+0j)
1010000001001	9	0	3		False	(-881.9999999999993+0j)
1100000001011	11	0	3		False	(-881.9999999999993+0j)
0100100110011	6.0	12.0	1		False	(-881.9999999999993+0j)
0000110101011	15.0	16.0	2		False	(-881.9999999999993+0j)
0001000110100	7.0	3.0	2		False	(-880.9999999999995+0j)
0001001000110	21.0	15.0	1		False	(-880.9999999999995+0j)
0001001000010	17.0	15.0	1		False	(-880.9999999999991+0j)
1100001000000	12.0	12.0	2		False	(-879.9999999999999+0j)
0110000001101	13	0	3		False	(-879.9999999999995+0j)
0000010110101	7.0	4.0	2		False	(-879.9999999999995+0j)
0000100110110	7.0	4.0	2		False	(-879.9999999999995+0j)
0

1100010010011	13.0	20.0	1		False	(-861.9999999999995+0j)
1010010100101	11.0	20.0	1		False	(-861.9999999999995+0j)
1000110001100	18.0	6.0	2		False	(-861.9999999999995+0j)
0110110001011	11	0	3		False	(-861.9999999999995+0j)
1000110001000	14.0	6.0	2		False	(-861.9999999999993+0j)
0110110001001	9	0	3		False	(-861.9999999999993+0j)
1010100100010	7.0	20.0	1		False	(-861.9999999999993+0j)
1100010100011	9.0	20.0	1		False	(-861.9999999999991+0j)
0111000010100	15.0	19.0	1		False	(-860.9999999999998+0j)
0111000101000	15.0	19.0	1		False	(-860.9999999999998+0j)
0011100001101	17.0	8.0	2		False	(-860.9999999999998+0j)
0011010001100	16.0	8.0	2		False	(-860.9999999999998+0j)
0111000010000	11.0	19.0	1		False	(-860.9999999999995+0j)
0011010001000	12.0	8.0	2		False	(-860.9999999999995+0j)
0111000100100	11.0	19.0	1		False	(-860.9999999999995+0j)
0101010001110	16.0	8.0	2		False	(-860.9999999999993+0j)
0101010001010	12.0	8.0	2		False	(-860.9999999999993+0j)
0011100001001	13.0	8.0	2		False	(-860.9999999999993

1011100000111	7	0	3		False	(-834.9999999999995+0j)
1011010000110	6	0	3		False	(-834.9999999999995+0j)
1101100000101	5	0	3		False	(-834.9999999999991+0j)
0011000001001	16.0	11.0	1		False	(-834.9999999999991+0j)
1011010000010	2	0	3		False	(-834.9999999999991+0j)
0101000001011	16.0	11.0	1		False	(-834.9999999999991+0j)
1011100000011	3	0	3		False	(-834.9999999999991+0j)
1101010000100	4	0	3		False	(-834.9999999999991+0j)
1110010100001	7.0	20.0	1		False	(-833.9999999999995+0j)
0010010001010	16.0	12.0	1		False	(-833.9999999999995+0j)
0100010001100	16.0	12.0	1		False	(-833.9999999999995+0j)
0010100001011	16.0	12.0	1		False	(-833.9999999999995+0j)
1110100100010	7.0	20.0	1		False	(-833.9999999999995+0j)
1100110001010	10	0	3		False	(-833.9999999999995+0j)
1010110001000	8	0	3		False	(-833.9999999999995+0j)
0100100001101	16.0	12.0	1		False	(-833.9999999999995+0j)
1100110000110	6	0	3		False	(-833.9999999999991+0j)
1010110000100	4	0	3		False	(-833.9999999999991+0j)
0111010001010	10	0	3		False	(-832.9

...

1001011110101	11.0	6.0	2		False	(3611+0j)
1001101110110	12.0	6.0	2		False	(3611+0j)
1111000111101	16.0	3.0	2		False	(3611+0j)
1110100111111	16.0	4.0	2		False	(3612+0j)
1110010111110	16.0	4.0	2		False	(3612+0j)
1000111110111	13.0	6.0	2		False	(3612+0j)
0011101111000	12.0	8.0	2		False	(3612.999999999999+0j)
0101011111001	11.0	8.0	2		False	(3613+0j)
0101101111010	12.0	8.0	2		False	(3613+0j)
0011011110111	11.0	8.0	2		False	(3613+0j)
0001111011110	14	0	3		False	(3613+0j)
0010111111001	13.0	8.0	2		False	(3614+0j)
0100111111011	13.0	8.0	2		False	(3614+0j)
1011001110011	6.0	3.0	2		False	(3615+0j)
1101001110101	8.0	3.0	2		False	(3615+0j)
1100011110110	8.0	4.0	2		False	(3616+0j)
1010101110101	6.0	4.0	2		False	(3616+0j)
1010011110100	6.0	4.0	2		False	(3616+0j)
1100101110111	8.0	4.0	2		False	(3616+0j)
0111001110111	10.0	3.0	2		False	(3617+0j)
0110011111000	10.0	4.0	2		False	(3617.999999999999+0j)
0110101111001	10.0	4.0	2		False	(3618+0j)
1110001011000	8	0	3		False	(3623.999999999999+0j)
1110001101

0110101011111	16.0	4.0	2		False	(4422+0j)
1010001011100	12	0	3		False	(4435.999999999999+0j)
1100001011110	14	0	3		False	(4436+0j)
1011101011000	8	0	3		False	(4442.999999999999+0j)
1101011011001	9	0	3		False	(4443+0j)
1101101101110	14	0	3		False	(4443+0j)
1101011101101	13	0	3		False	(4443+0j)
1011011101011	11	0	3		False	(4443+0j)
1101101011010	10	0	3		False	(4443+0j)
1011011010111	7	0	3		False	(4443+0j)
1011101101100	12	0	3		False	(4443+0j)
1100111101111	15	0	3		False	(4444+0j)
1010111101101	13	0	3		False	(4444+0j)
1100111011011	11	0	3		False	(4444+0j)
1010111011001	9	0	3		False	(4444+0j)
0111101011100	12	0	3		False	(4444.999999999999+0j)
0111011011011	11	0	3		False	(4445+0j)
0111011101111	15	0	3		False	(4445+0j)
0110111011101	13	0	3		False	(4445.999999999999+0j)
1011010111111	15	0	3		False	(4447+0j)
1111001001111	30.0	15.0	1		False	(4451+0j)
1001001111001	18.0	9.0	1		False	(4467+0j)
1000011111010	18.0	10.0	1		False	(4468+0j)
1000101111011	18.0	10.0	1		False	(4468+0j)
0101001111101	18.

1111001110100	7.0	3.0	2		False	(5033+0j)
1110011110101	7.0	4.0	2		False	(5034+0j)
1110101110110	7.0	4.0	2		False	(5034+0j)
0001111111101	13	0	3		False	(5038.999999999999+0j)
1111111010101	5	0	3		False	(5045+0j)
1111111101001	9	0	3		False	(5045+0j)
1111110111101	13	0	3		False	(5048.999999999999+0j)
1110001110111	7	0	3		False	(5050+0j)
1101111110101	5	0	3		False	(5053+0j)
1011111110011	3	0	3		False	(5053+0j)
0111111110111	7	0	3		False	(5055+0j)
1111011110010	2	0	3		False	(5057+0j)
1111101110011	3	0	3		False	(5057+0j)
1110111110100	4	0	3		False	(5058+0j)
1111111110001	1	0	3		False	(5121+0j)
1001001011111	24.0	9.0	1		False	(5351+0j)
1001011011101	19.0	6.0	2		False	(5374.999999999999+0j)
1001101011110	20.0	6.0	2		False	(5375+0j)
1000111011111	21.0	6.0	2		False	(5376+0j)
0011011011111	19.0	8.0	2		False	(5377+0j)
1101001011101	16.0	3.0	2		False	(5378.999999999999+0j)
1011001101111	18.0	3.0	2		False	(5379+0j)
1011001011011	14.0	3.0	2		False	(5379+0j)
1010101011101	14.0	4.0	2		False	(5379.99999

1010011111010	12.0	4.0	2		False	(6496+0j)
1010101111011	12.0	4.0	2		False	(6496+0j)
0111001111101	16.0	3.0	2		False	(6496.999999999999+0j)
0110011111110	16.0	4.0	2		False	(6498+0j)
0110101111111	16.0	4.0	2		False	(6498+0j)
1110001011110	14	0	3		False	(6504+0j)
1101111011100	12	0	3		False	(6506.999999999999+0j)
1011111101110	14	0	3		False	(6507+0j)
1011111011010	10	0	3		False	(6507+0j)
0111111011110	14	0	3		False	(6509+0j)
1111011101101	13	0	3		False	(6511+0j)
1111101101110	14	0	3		False	(6511+0j)
1111011011001	9	0	3		False	(6511+0j)
1111101011010	10	0	3		False	(6511+0j)
1010001111100	12	0	3		False	(6511.999999999999+0j)
1100001111110	14	0	3		False	(6512+0j)
1110111011011	11	0	3		False	(6512+0j)
1110111101111	15	0	3		False	(6512+0j)
1011101111000	8	0	3		False	(6518.999999999999+0j)
1101101111010	10	0	3		False	(6519+0j)
1101011111001	9	0	3		False	(6519+0j)
1011011110111	7	0	3		False	(6519+0j)
1010111111001	9	0	3		False	(6520+0j)
1100111111011	11	0	3		False	(6520+0j)
0111101111100	12	0	3	

1101011111111	15	0	3		False	(10119+0j)
1010111111111	15	0	3		False	(10120+0j)
1111001111101	16.0	3.0	2		False	(10162.999999999998+0j)
1110101111111	16.0	4.0	2		False	(10164+0j)
1110011111110	16.0	4.0	2		False	(10164+0j)
1111111011110	14	0	3		False	(10175+0j)
1011111111100	12	0	3		False	(10182.999999999998+0j)
1101111111110	14	0	3		False	(10183+0j)
1111101111100	12	0	3		False	(10186.999999999998+0j)
1111011111011	11	0	3		False	(10187+0j)
1110111111101	13	0	3		False	(10187.999999999998+0j)
1111111111010	10	0	3		False	(10251+0j)
1011011111110	14	0	3		False	(10789+0j)
1011101111111	15	0	3		False	(10789+0j)
1111001111110	17.0	3.0	2		False	(10833+0j)
1110011111111	17.0	4.0	2		False	(10834+0j)
1111111011111	15	0	3		False	(10845+0j)
1011111111101	13	0	3		False	(10852.999999999998+0j)
1101111111111	15	0	3		False	(10853+0j)
1111101111101	13	0	3		False	(10856.999999999998+0j)
1111011111100	12	0	3		False	(10856.999999999998+0j)
1110111111110	14	0	3		False	(10858+0j)
1111111111011	11	0	3		False	(10

In [11]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [12]:
mixer = SummedOp([
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ X ^ X,
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ Y ^ Y,
    
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ X ^ X ^ I,
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ Y ^ Y ^ I,
    
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ X ^ X    ^ I ^ I ^ I,
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ Y ^ Y    ^ I ^ I ^ I,
    
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ X ^ X ^ I    ^ I ^ I ^ I,
    I ^ I ^ I ^ I    ^ I ^ I ^ I    ^ Y ^ Y ^ I    ^ I ^ I ^ I,
    
    I ^ I ^ I ^ I    ^ I ^ X ^ X    ^ I ^ I ^ I    ^ I ^ I ^ I,
    I ^ I ^ I ^ I    ^ I ^ Y ^ Y    ^ I ^ I ^ I    ^ I ^ I ^ I,
    
    I ^ I ^ I ^ I    ^ X ^ X ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I,
    I ^ I ^ I ^ I    ^ Y ^ Y ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I,
    
    I ^ I ^ I ^ X    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I,
    I ^ I ^ X ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I,
    I ^ X ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I,
    X ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I    ^ I ^ I ^ I,
])

In [ ]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, mixer=mixer) #TODO: initial state

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)